In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import timm
import numpy as np
from torchvision import transforms
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

Using device: cuda


In [2]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.1):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class CPEA(nn.Module):
    def __init__(self, in_dim=384):
        super().__init__()
        self.fc1 = Mlp(in_features=in_dim, hidden_features=int(in_dim/4), out_features=in_dim)
        self.fc_norm1 = nn.LayerNorm(in_dim)
        self.fc2 = Mlp(in_features=196**2, hidden_features=256, out_features=1)
        self.class_agnostic_embedding = nn.Parameter(torch.randn(1, 1, in_dim))

    def forward(self, feat_query, feat_shot, shot):
        B, N, C = feat_query.size()

        # Generate class-aware embeddings
        feat_query = self.fc1(torch.mean(feat_query, dim=1, keepdim=True)) + feat_query
        feat_shot = self.fc1(torch.mean(feat_shot, dim=1, keepdim=True)) + feat_shot

        feat_query = self.fc_norm1(feat_query)
        feat_shot = self.fc_norm1(feat_shot)

        # Split class token and patch embeddings
        query_class = feat_query[:, 0, :].unsqueeze(1)
        query_patches = feat_query[:, 1:, :]

        support_class = feat_shot[:, 0, :].unsqueeze(1)
        support_patches = feat_shot[:, 1:, :]

        # Class-aware patch embedding adaptation
        feat_query = query_patches + 2.0 * query_class
        feat_shot = support_patches + 2.0 * support_class

        # Normalization
        feat_query = F.normalize(feat_query, p=2, dim=2)
        feat_query = feat_query - torch.mean(feat_query, dim=2, keepdim=True)

        # Reshape support features
        feat_shot = feat_shot.contiguous().reshape(shot, -1, N-1, C)
        feat_shot = feat_shot.mean(dim=0)
        feat_shot = F.normalize(feat_shot, p=2, dim=2)
        feat_shot = feat_shot - torch.mean(feat_shot, dim=2, keepdim=True)

        # Compute similarity scores
        results = []
        for idx in range(feat_query.size(0)):
            tmp_query = feat_query[idx].unsqueeze(0)
            out = torch.matmul(feat_shot, tmp_query.transpose(1, 2))
            out = out.flatten(1)
            out = self.fc2(out.pow(2))
            out = out.transpose(0, 1)
            results.append(out)

        return torch.cat(results, dim=0)

class CPEAVisionTransformer(nn.Module):
    def __init__(self, pretrained_model='vit_small_patch16_224'):
        super().__init__()
        self.vit = timm.create_model(pretrained_model, pretrained=True, num_classes=0)
        self.embed_dim = self.vit.embed_dim
        self.cpea = CPEA(in_dim=self.embed_dim)

    def forward_features(self, x):
        x = self.vit.patch_embed(x)
        cls_token = self.vit.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        x = self.vit.pos_drop(x + self.vit.pos_embed)

        for blk in self.vit.blocks:
            x = blk(x)
        x = self.vit.norm(x)
        return x

    def forward(self, support_images, query_images, n_way, n_support, n_query):
        support_features = self.forward_features(support_images.view(-1, *support_images.shape[2:]))
        query_features = self.forward_features(query_images.view(-1, *query_images.shape[2:]))

        logits = self.cpea(query_features, support_features, n_support)
        return logits

In [3]:
def create_cifar_fs():
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    cifar100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

    # Split classes for CIFAR-FS
    base_classes = np.random.choice(100, 64, replace=False)
    cifar_fs_indices = np.where(np.isin(cifar100.targets, base_classes))[0]
    cifar_fs = torch.utils.data.Subset(cifar100, cifar_fs_indices)
    cifar_fs_targets = np.array(cifar100.targets)[cifar_fs_indices]

    return cifar_fs, cifar_fs_targets

def create_fewshot_task(dataset, targets, n_way, n_support, n_query):
    classes = np.random.choice(np.unique(targets), n_way, replace=False)
    support_set = []
    query_set = []

    for cls in classes:
        cls_indices = np.where(targets == cls)[0]
        support_indices = np.random.choice(cls_indices, n_support, replace=False)
        query_indices = np.random.choice(np.setdiff1d(cls_indices, support_indices), n_query, replace=False)

        support_set.append(torch.stack([dataset[i][0] for i in support_indices]))
        query_set.append(torch.stack([dataset[i][0] for i in query_indices]))

    return torch.stack(support_set), torch.stack(query_set)

In [4]:
def train_episode(model, optimizer, dataset, targets, n_way, n_support, n_query, device):
    model.train()
    optimizer.zero_grad()

    support_images, query_images = create_fewshot_task(dataset, targets, n_way, n_support, n_query)
    support_images, query_images = support_images.to(device), query_images.to(device)

    logits = model(support_images, query_images, n_way, n_support, n_query)
    loss = F.cross_entropy(logits, torch.arange(n_way).repeat_interleave(n_query).to(device))

    loss.backward()
    optimizer.step()

    return loss.item()

def train(model, dataset, targets, n_episodes, n_way, n_support, n_query, device):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5)

    for episode in tqdm(range(n_episodes)):
        loss = train_episode(model, optimizer, dataset, targets, n_way, n_support, n_query, device)
        scheduler.step()

        if (episode + 1) % 100 == 0:
            print(f"Episode {episode + 1}, Loss: {loss:.4f}")

    return model

def evaluate(model, dataset, targets, n_episodes, n_way, n_support, n_query, device):
    model.eval()
    accuracies = []

    for _ in tqdm(range(n_episodes)):
        support_images, query_images = create_fewshot_task(dataset, targets, n_way, n_support, n_query)
        support_images, query_images = support_images.to(device), query_images.to(device)

        with torch.no_grad():
            logits = model(support_images, query_images, n_way, n_support, n_query)
            _, predicted = torch.max(logits, 1)
            correct = (predicted == torch.arange(n_way).repeat_interleave(n_query).to(device)).sum().item()
            accuracies.append(correct / (n_way * n_query))

    return np.mean(accuracies)

In [5]:
# Create dataset
cifar_fs, cifar_fs_targets = create_cifar_fs()

# Initialize model
model = CPEAVisionTransformer().to(device)

# Training parameters (as per paper)
n_episodes = 10000  # Paper uses 10000 episodes
n_way = 5
n_support = 5
n_query = 15

# Train the model
model = train(model, cifar_fs, cifar_fs_targets, n_episodes, n_way, n_support, n_query, device)

# Save the model
torch.save({
    'model_state_dict': model.state_dict(),
    'n_way': n_way,
    'n_support': n_support,
    'n_query': n_query
}, 'cpea_model.pth')

# Evaluate
eval_episodes = 600  # Paper evaluates on 600 episodes
accuracy = evaluate(model, cifar_fs, cifar_fs_targets, eval_episodes, n_way, n_support, n_query, device)
print(f"5-way 5-shot accuracy: {accuracy:.4f}")

100%|██████████| 169M/169M [00:02<00:00, 78.5MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

  1%|          | 100/10000 [02:25<3:57:56,  1.44s/it]

...

Episode 9900, Loss: 1.6094


100%|██████████| 10000/10000 [3:57:07<00:00,  1.42s/it]

Episode 10000, Loss: 1.6094



100%|██████████| 600/600 [06:21<00:00,  1.57it/s]

5-way 5-shot accuracy: 0.1988


In [7]:
def load_model(path):
    checkpoint = torch.load(path)
    model = CPEAVisionTransformer().to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model, checkpoint['n_way'], checkpoint['n_support'], checkpoint['n_query']

# Load and evaluate
loaded_model, n_way, n_support, n_query = load_model('cpea_model.pth')
accuracy = evaluate(loaded_model, cifar_fs, cifar_fs_targets, eval_episodes, n_way, n_support, n_query, device)
print(f"Loaded model accuracy: {accuracy:.4f}")

<ipython-input-7-3ba8a8353493>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)
100%|██████████| 600/600 [06:24<00:00,  1.56it/s]

Loaded model accuracy: 0.1986
